## Prepare data for ML


We will take the output from thee **Clean_data** analysis and convert the category feature into dummy varaibles
output will be:
- Split the training set into 80/20 trainning and validation
- Take the whole testing(without labels) as testing dataset

In [1]:
import pandas as pd
import numpy as np
import pickle


In [2]:
def make_features(d):
    category_columns = d.dtypes[d.dtypes == object]
    return pd.get_dummies(d, columns=category_columns.index)

In [3]:
dTrain = pd.read_csv("../data/train_clean.csv")
dTest = pd.read_csv("../data/test_clean.csv")
n = dTrain.shape[0] ## total rows of training
d = make_features(dTrain.append(dTest))
d = d.astype(float)

## Testing and training
dTrain_transformed = d.iloc[:n]
dTest_transformed = d.iloc[n:]


In [4]:
## Read Training labels
LTrain = pd.read_csv("../data/train_label.csv").iloc[:,1]
LTrain = LTrain.astype("category")
LTrain_coded = np.array(LTrain.cat.codes.tolist())

In [5]:
idTest = pd.read_csv("../data/test_id.csv")  ## dataFrame

In [6]:
## Instead of using sklean.model_selection.train_test_split,
## We use numpy to resampling
np.random.seed(123)
training_idx = np.random.randint(dTrain_transformed.shape[0],size=int(dTrain_transformed.shape[0]*0.8))
validation_idx = np.random.randint(dTrain_transformed.shape[0],size=int(dTrain_transformed.shape[0]*0.2))

data_train,data_validation = dTrain_transformed.as_matrix()[training_idx],dTrain_transformed.as_matrix()[validation_idx]
labels_train,labels_validation = LTrain_coded[training_idx],LTrain_coded[validation_idx]
labels_category_train,labels_category_validation = LTrain[training_idx],LTrain[validation_idx]

In [7]:
## Write out by pickles
training = {"features": data_train,  "label":labels_train,"label_original":labels_category_train}
validating = {"features": data_validation,  "label":labels_validation,"label_original":labels_category_validation}
testing = {"features": dTest_transformed.as_matrix(),  "id":idTest}

outd = {"train":training,"validation":validating, "test":testing,}

with open('../data/features.pickle', 'wb') as handle:
    pickle.dump(outd, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [8]:
#find the names of the imported modules
import types
def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__

#exclude all modules not listed by `!pip freeze`
excludes = ['__builtin__', 'types', 'IPython.core.shadowns', 'sys', 'os']
imported_modules = [module for module in imports() if module not in excludes]
pip_modules = !pip freeze #you could also use `!conda list` with anaconda

#print the names and versions of the imported modules
for module in pip_modules:
    name, version = module.split('==')
    if name in imported_modules:
        print(name + '\t' + version)

numpy	1.13.3
pandas	0.20.3
